<a href="https://colab.research.google.com/github/Pankaj122002/Python/blob/main/Text_Analysis(Web_scraping).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install requests
!pip install beautifulsoup4
!pip install textblob
!pip install syllables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-resources
    Found existing installation: importlib-resources 6.0.1
    Uninstalling importlib-resources-6.0.1:
      Successfully uninstalled importlib-resources-6.0.1
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.8.0
    Uninstalling importlib-metadata-6.8.0:
      Successfully uninstalled importlib-metadata-6.8.0


In [4]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
input_df = pd.read_excel('/content/Input.xlsx')

# Initialize a list to store analysis results

results = []

# Iterate through each row in the input DataFrame
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    def extract_article_text(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

    # Find and extract the article title and text
        article_text = ""
        article_title = soup.find('title').get_text()
        paragraphs = soup.find_all(['p', 'div'])  # Adjust based on the actual structure
        for p in paragraphs:
            article_text += p.get_text()
        return article_title, article_text
    article_title, article_text = extract_article_text(url)
    # Perform text analysis using TextBlob
    b = TextBlob(article_text)
    positive_score = b.sentiment.polarity
    negative_score = b.sentiment.polarity
    polarity_score = b.sentiment.polarity
    subjectivity_score = b.sentiment.subjectivity
    avg_sentence_length = len(b.words) / len(b.sentences)
    import re
    def count_syllables(word):
    # Convert the word to lowercase
        word = word.lower()

    # Define a regular expression to match consecutive vowels
    # but not ending with 'e'
        pattern = '(?!e$)[aeiouy]+'

    # Find all matches of the pattern in the word
        matches = re.findall(pattern, word, re.I)

    # Return the count of matches (syllables)
        return len(matches)


    word = "syllable"
    syllable_count = count_syllables(word)
    def count_complex_words(b):
        complex_word_count = sum(1 for word in b.words if syllable_count >= 3)
        return complex_word_count
    complex_word_count = count_complex_words(b)
    percentage_complex_words = (complex_word_count / len(b.words)) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_words_per_sentence = len(b.words) / len(b.sentences)
    word_count = len(b.words)
    avg_syllables_per_word = syllable_count / word_count
    personal_pronoun_count = sum(1 for word in b.words if word.lower() in ['i', 'you', 'he', 'she', 'we', 'they'])
    avg_word_length = sum(len(word) for word in b.words) / word_count


    result = {
        'URL_ID': url_id,
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX':fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE' :avg_words_per_sentence,
        'COMPLEX WORD COUNT': complex_word_count,
        'WORD COUNT': word_count,
        'SYLLABLE PER WORD': avg_syllables_per_word,
        'PERSONAL PRONOUNS': personal_pronoun_count,
        'AVG WORD LENGTH': avg_word_length
          }
    results.append(result)
output_df = pd.DataFrame(results)
output_df.to_excel('output.xlsx', index=False)

In [17]:
print(output_df)

      URL_ID  POSITIVE SCORE  NEGATIVE SCORE  POLARITY SCORE  \
0      123.0        0.143031        0.143031        0.143031   
1      321.0        0.138799        0.138799        0.138799   
2     2345.0        0.112002        0.112002        0.112002   
3     4321.0        0.150462        0.150462        0.150462   
4      432.0        0.144979        0.144979        0.144979   
..       ...             ...             ...             ...   
109  50921.0        0.122659        0.122659        0.122659   
110  51382.8        0.073174        0.073174        0.073174   
111  51844.6        0.145176        0.145176        0.145176   
112  52306.4        0.116704        0.116704        0.116704   
113  52768.2        0.115631        0.115631        0.115631   

     SUBJECTIVITY SCORE  AVG SENTENCE LENGTH  PERCENTAGE OF COMPLEX WORDS  \
0              0.384337            37.326121                          0.0   
1              0.419228            62.300366                          0.0   


# New Section